# Final Masters Project

## Name: Sreekanth Palagiri, Student ID: R00184198

## Project Topic: Evaluation of Ensemble Approach for Sentiment Analysis on a Small Dataset

##NoteBook1: Trainer Flair


### **Mount google drive**

In [1]:
from google.colab import drive 
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
!ls "gdrive/My Drive/Colab Notebooks/Masters Project"

'Airline Tweets dataset'  'Sentence Polarity Dataset'
 glove.6B.300d.txt	   VMDataset


### **Install Flair**

In [3]:
!pip install flair

     |████████████████████████████████| 286kB 19.0MB/s 
     |████████████████████████████████| 2.1MB 18.7MB/s 
     |████████████████████████████████| 71kB 11.5MB/s 
     |████████████████████████████████| 983kB 45.8MB/s 
     |████████████████████████████████| 798kB 45.6MB/s 
     |████████████████████████████████| 1.2MB 47.4MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 776.8MB 22kB/s 
     |████████████████████████████████| 19.7MB 1.3MB/s 
     |████████████████████████████████| 901kB 51.3MB/s 
     |████████████████████████████████| 3.3MB 43.0MB/s 
  Created wheel for gdown: filename=gdown-3.12.2-cp37-none-any.whl size=9693 sha256=9b4b97c16192e5e3c40af3bb8dd75184e191054ce78aba07c7c9391ffed1936d
  Stored in directory: /root/.cache/pip/wheels/81/d0/d7/d9983facc6f2775411803e0e2d30ebf98efbf2fc6e57701e09
Successfully built gdown
  Created wheel for ftfy: filename

### **Load Data and Preprocess**

In [4]:
import pandas as pd
import numpy as np

df=pd.read_csv("/content/gdrive/My Drive/Colab Notebooks/Masters Project/VMDataset/Export_loop-sentiment-pos-neg-train_05112020000000.csv")
print(df.groupby(['label']).size())
df.head()

label
Negative     887
Positive    1013
dtype: int64


,label,text
0,Negative,No one cares about marketing slides - a techni...
1,Positive,Are all three hosts providing storage/capacity...
2,Negative,would loved to had managed to get down to the ...
3,Negative,Vending machine at work is out of Dasani water...
4,Positive,"RT @VMwareEdu: Paul Maritz, CEO and President ..."


**Preprocessor to Remove all special characters except emoticons**

In [5]:
import re

def preprocessor(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    text = re.sub('[^A-Za-z0-9\']+', ' ', text.lower()) +\
        ' '.join(emoticons).replace('-', '')
    return text

print(df['text'][19])
print(preprocessor(df['text'][19]))

Kristina,  Any updates from your side ? I volunteer for beta test :)  - really need that app running as workaround are driving me nuts ....
kristina any updates from your side i volunteer for beta test really need that app running as workaround are driving me nuts :)


In [6]:
df['text'] = df['text'].apply(preprocessor)
df1=df.copy()

**Label Encoding to the format specified by Flair**

In [7]:
df['label'] = '__label__' + df['label'].astype(str)
df.head()

,label,text
0,__label__Negative,no one cares about marketing slides a technica...
1,__label__Positive,are all three hosts providing storage capacity...
2,__label__Negative,would loved to had managed to get down to the ...
3,__label__Negative,vending machine at work is out of dasani water...
4,__label__Positive,rt vmwareedu paul maritz ceo and president of ...


In [8]:
from sklearn.model_selection import train_test_split

df_train, df_test, sentiment_train, sentiment_test = train_test_split(df1['text'], df1['label'], 
                                                                      random_state=1, test_size=0.15, 
                                                                      shuffle=False)


print('Length of train set:',len(df_train),'Length of test set:',len(df_test))

Length of train set: 1615 Length of test set: 285


**Flair Needs validation, test and train data in csv's.**

**Creating csv files with 85-10-5 split.**

In [9]:
df.iloc[0:int(len(df)*0.85)].to_csv('/content/gdrive/My Drive/Colab Notebooks/Masters Project/VMDataset/Models/train.csv', sep='\t', 
                                                index = False, header = False)

df.iloc[int(len(df)*0.85):int(len(df)*0.95)].to_csv('/content/gdrive/My Drive/Colab Notebooks/Masters Project/VMDataset/Models/test.csv', sep='\t', 
                                                                      index = False, header = False)

df.iloc[int(len(df)*0.95):].to_csv('/content/gdrive/My Drive/Colab Notebooks/Masters Project/VMDataset/Models/val.csv', sep='\t', 
                                               index = False, header = False);

**Creating corpus with Train, Test and Validation data for model training.**

In [10]:
from flair.data import Corpus
from flair.datasets import ClassificationCorpus
#from flair.data import Sentence

data_folder = '/content/gdrive/My Drive/Colab Notebooks/Masters Project/VMDataset/Models/'

# this is the folder in which training, test and dev files reside

corpus: Corpus = ClassificationCorpus(data_folder,
                                      test_file='test.csv',
                                      dev_file='val.csv',
                                      train_file='train.csv')

2021-04-29 21:28:05,062 Reading data from /content/gdrive/My Drive/Colab Notebooks/Masters Project/VMDataset/Models
2021-04-29 21:28:05,063 Train: /content/gdrive/My Drive/Colab Notebooks/Masters Project/VMDataset/Models/train.csv
2021-04-29 21:28:05,068 Dev: /content/gdrive/My Drive/Colab Notebooks/Masters Project/VMDataset/Models/val.csv
2021-04-29 21:28:05,070 Test: /content/gdrive/My Drive/Colab Notebooks/Masters Project/VMDataset/Models/test.csv


### **Traing Flair Model**


In [11]:
from torch.optim.adam import Adam
from flair.data import Corpus
#from flair.datasets import TREC_6
from flair.embeddings import TransformerDocumentEmbeddings
from flair.models import TextClassifier
from flair.trainers import ModelTrainer


# 2. create the label dictionary
label_dict = corpus.make_label_dictionary()

# 3. initialize transformer document embeddings (many models are available)
document_embeddings = TransformerDocumentEmbeddings('distilbert-base-uncased', fine_tune=True)

# 4. create the text classifier
classifier = TextClassifier(document_embeddings, label_dictionary=label_dict)

# 5. initialize the text classifier trainer with Adam optimizer
trainer = ModelTrainer(classifier, corpus, optimizer=Adam)

# 6. start the training
trainer.train('/content/gdrive/My Drive/Colab Notebooks/Masters Project/VMDataset/Models/resources/taggers/trec',
              learning_rate=3e-5, # use very small learning rate
              mini_batch_size=16,
              mini_batch_chunk_size=4, # optionally set this if transformer is too much for your machine
              max_epochs=5, # terminate after 5 epochs
              )

2021-04-29 21:28:05,107 Computing label dictionary. Progress:


100%|██████████| 1805/1805 [00:01<00:00, 1370.56it/s]

2021-04-29 21:28:06,534 [b'Negative', b'Positive']



2021-04-29 21:28:17,017 ----------------------------------------------------------------------------------------------------
2021-04-29 21:28:17,023 Model: "TextClassifier(
  (document_embeddings): TransformerDocumentEmbeddings(
    (model): DistilBertModel(
      (embeddings): Embeddings(
        (word_embeddings): Embedding(30522, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (transformer): Transformer(
        (layer): ModuleList(
          (0): TransformerBlock(
            (attention): MultiHeadSelfAttention(
              (dropout): Dropout(p=0.1, inplace=False)
              (q_lin): Linear(in_features=768, out_features=768, bias=True)
              (k_lin): Linear(in_features=768, out_features=768, bias=True)
              (v_lin): Linear(in_features=768, out_features=768, bias=True)
              (out_lin): Linear(i

{'dev_loss_history': [0.32350751757621765,
  0.5357191562652588,
  0.7196952700614929,
  0.8626624345779419,
  1.0973896980285645],
 'dev_score_history': [0.8526, 0.7895, 0.8526, 0.8632, 0.8526],
 'test_score': 0.8684,
 'train_loss_history': [0.47213924484382763,
  0.203430224458415,
  0.060567354186976376,
  0.026288604924162642,
  0.02858893608257485]}

In [22]:
from flair.data import Sentence

model_flair=TextClassifier.load('/content/gdrive/My Drive/Colab Notebooks/Masters Project/VMDataset/Models/resources/taggers/trec/best-model.pt')

results=[]
Y_pred=[]
for i in df_test.index:
    sentence=Sentence(df_test[i])
    model_flair.predict(sentence)
    if sentence.get_labels()[0].value=='Positive':
      score=1-sentence.get_labels()[0].score
      Y_pred.append('Positive')
    else:
      score=sentence.get_labels()[0].score
      Y_pred.append('Negative')
    results.append([score,1-score])

2021-04-29 21:50:10,130 loading file /content/gdrive/My Drive/Colab Notebooks/Masters Project/VMDataset/Models/resources/taggers/trec/best-model.pt


In [28]:
from sklearn import metrics

Y_test=sentiment_test
print('F1 Score:',metrics.f1_score(Y_test,Y_pred,pos_label='Positive'),
      'Precision:',metrics.precision_score(Y_test,Y_pred,pos_label='Positive'),
      'Recall:',metrics.recall_score(Y_test,Y_pred,pos_label='Positive'),
      'Accuracy:',metrics.accuracy_score(Y_test,Y_pred))

F1 Score: 0.8812500000000001 Precision: 0.8597560975609756 Recall: 0.9038461538461539 Accuracy: 0.8666666666666667
